## 0.0 Notebook Statement

This will be the notebook for the final model to bring together all data in a final optimsaition

The model in this notebook will work off the ernegy price, energy consumption and energy generation models with one week of data

The model will predict three days of hourly energy storage

# 1.0 Imports

In [37]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.optimize import Bounds


# 2.0 Generation of practice data

In [38]:
# create sales data. Scale between 0.5 and 1
sell = np.random.random((72,1))/5+0.2
sell_price = pd.DataFrame(data=sell, columns=['SalePrice_£/kwh'])

In [39]:
# create purchase data. Use a two time sell price multiplier
purchase_price = sell_price.rename(columns={'SalePrice_£/kwh':'PurchasePrice_£/kwh'})
purchase_price['PurchasePrice_£/kwh'] = purchase_price['PurchasePrice_£/kwh'].apply(lambda x: x*2)

In [40]:
# create generation data. Average of 0.1 kwh per hour.
generation = np.random.random((72,1))/2
generation = pd.DataFrame(data=generation, columns=['Generation_kwh'])

In [41]:
# create comsumption data. Average of 0.3 kwh per hour.
consumption = np.random.random((72,1))/5+0.2
consumption = pd.DataFrame(data=consumption, columns=['Consumption_kwh'])

In [42]:
# combine all data
df = pd.concat([sell_price, purchase_price, generation, consumption], axis=1)

In [43]:
df.head(15)

,SalePrice_£/kwh,PurchasePrice_£/kwh,Generation_kwh,Consumption_kwh
0,0.350348,0.700696,0.333470,0.300905
1,0.245152,0.490305,0.070384,0.274157
2,0.313164,0.626328,0.061990,0.360037
3,0.281366,0.562733,0.096241,0.382336
4,0.242793,0.485587,0.467314,0.235215
5,0.356433,0.712867,0.484503,0.289666
6,0.376478,0.752955,0.233296,0.328824
7,0.278191,0.556382,0.443396,0.365215
8,0.298817,0.597634,0.118079,0.278313
9,0.362085,0.724170,0.114780,0.264336


# 3.0 Optimisation model

In [44]:
data = np.array(df)

In [45]:
def profit(x_input):
    '''
    Function to be minimised for the optimsation problem
    '''
    x0 = x_input[0:72]
    x1 = x_input[72:]

    battery = np.zeros(73)
    # initial battery charge
    battery[0] = 1
    # battery size
    battery_size = 5
    cost_punishment = 0
    for i in range(len(battery)-1):
        battery[i+1] = battery[i] + data[i,2] - data[i,3] + x0[i] - x1[i]
        if battery[i + 1] > battery_size:
            cost_punishment += 1000
        if battery[i+1] < 0:
            cost_punishment += 1000

    buy = x0[:] * data[:,1]
    sell = x1[:] * data[:,0]


    cost = np.sum(buy - sell) + cost_punishment
    battery_charge = battery[72] * np.mean(data[i,0])
    return cost - battery_charge

In [46]:
# x0 = initial purchase amount
x0 = np.array(df['Consumption_kwh'])
# x1 = initial sale amount
x1 =  np.array(df['Generation_kwh'])
x_input = np.concatenate((x0,x1),axis=0)
cost = profit(x_input)
cost

7.379524929133122

In [47]:
x_input = np.concatenate((x0,x1),axis=0)

res = minimize(
    profit,
    x_input,
    method='nelder-mead',
    options={'xatol': 1e-8, 'disp': True}
    )
res.x

/tmp/ipykernel_339944/3550192192.py:3: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  res = minimize(


array([-1.45119456e-01,  2.52836650e-01,  5.18465311e-01,  1.33241648e-02,
        3.28487430e-01,  5.19777766e-01,  8.63246855e-02,  6.04647482e-01,
        3.41102372e-01, -3.30936718e-01,  4.29669828e-02,  4.79916982e-01,
        7.54050412e-01,  5.24150702e-01,  2.49747022e-01, -1.20971977e-01,
        6.81390660e-01, -3.26736223e-01,  3.81859372e-01,  4.17815929e-01,
       -2.32429048e-01,  1.25759067e-01, -5.26463372e-01,  4.52718034e-01,
        1.18231043e+00,  1.51974536e+00,  3.09399677e-01,  3.37254298e-01,
        1.54314022e-01,  2.29075318e-01, -4.77415167e-02,  2.44025762e-01,
        1.07290431e+00, -2.77589368e-03,  2.61181907e-01,  4.27784337e-02,
       -4.41822932e-01,  2.78473897e-01,  3.64907019e-01,  9.81814605e-01,
        4.12108813e-01,  4.54630206e-01,  1.35920197e+00,  7.34857030e-01,
        7.88645144e-01,  1.73339554e-01,  6.34264121e-01,  6.79663810e-01,
        1.23464187e+00,  1.68444817e-01,  5.31282337e-01, -4.21774145e+00,
        7.29050583e-01,  

In [48]:
best_case = profit(res.x)
best_case

4.008397478925271

### Inputting bounds

In [49]:
#class scipy.optimize.Bounds(lb=-inf, ub=inf, keep_feasible=False)

In [50]:
# x0 is the energy purchased
# x1 is the energy sold
# lower bound for x0 is 0, upper bound is 3 (assumptino set from grid)
# lower bound for x1 is 0, upper bound is the PV energy generation
lb =np.concatenate((np.ones(72)*0, np.ones(72)*0),axis = 0)
ub =np.concatenate((np.ones(72)*3, data[:,2]), axis = 0)
bounds = Bounds(lb=lb, ub=ub)

In [51]:
x_input = np.concatenate((x0,x1),axis=0)

res = minimize(
    profit,
    x_input,
    bounds = bounds,
    method='nelder-mead',
    options={'xatol': 1e-12, 'disp': True}
    )
res.x

/tmp/ipykernel_339944/542866697.py:3: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  res = minimize(


array([1.28480353e-01, 2.36323438e-01, 3.70937812e-01, 5.90802760e-01,
       7.16687828e-01, 4.25539373e-02, 4.42561632e-01, 4.48771580e-01,
       7.82366011e-01, 7.80068496e-02, 1.71433857e-02, 2.82410403e-01,
       3.71372945e-01, 5.83289985e-01, 2.37818475e-01, 5.58798654e-01,
       7.49290624e-01, 1.60062569e-02, 1.49980567e+00, 4.54097762e-01,
       6.90077670e-01, 1.32957751e-01, 2.57298350e-01, 3.70779432e-01,
       9.31365936e-03, 3.25549477e-01, 5.24570574e-02, 1.95351813e-02,
       8.47966571e-02, 2.66187830e-02, 4.38685138e-02, 4.94422947e-01,
       4.96951795e-01, 6.83318905e-02, 2.39267325e-02, 1.40053712e-01,
       2.80953870e-02, 6.27354953e-01, 3.85276241e-02, 9.08181474e-02,
       3.47781801e-01, 1.43973155e-01, 5.59528593e-01, 4.91822792e-01,
       1.20718430e-02, 9.42168640e-03, 1.04423134e-01, 4.60951627e-01,
       1.17005186e-01, 6.47711369e-02, 3.61349994e-01, 3.54125776e-02,
       3.00256769e-01, 9.53861316e-02, 8.25339591e-01, 3.91364989e-01,
      

In [52]:
best_case = profit(res.x)
best_case

5.791831020484753

In [53]:
def battery_storage(x_input):
    '''
    Function to be minimised for the optimsation problem
    '''
    x0 = x_input[0:72]
    x1 = x_input[72:]

    battery = np.zeros(73)
    # initial battery charge
    battery[0] = 1
    # battery size
    battery_size = 5
    cost_punishment = 0
    for i in range(len(battery)-1):
        battery[i+1] = battery[i] + data[i,2] - data[i,3] + x0[i] - x1[i]
        if battery[i + 1] > battery_size:
            cost_punishment += 1000

    buy = x0[:] * data[:,1]
    sell = x1[:] * data[:,0]


    cost = np.sum(buy - sell) + cost_punishment
    battery_charge = battery[72] * np.mean(data[i,0])
    return battery, (cost - battery_charge)

In [54]:
(battery, profit) = battery_storage(res.x)
battery

array([1.        , 0.82757499, 0.78974143, 0.80064239, 1.00910964,
       1.49058218, 1.24346966, 1.35720748, 1.4407637 , 1.94481676,
       1.75848733, 1.4561564 , 1.4538603 , 1.58416802, 1.8825772 ,
       1.72835685, 2.01599083, 2.47057903, 2.19661403, 3.40491948,
       3.56005191, 3.96102952, 3.7705728 , 3.77252319, 3.78787292,
       3.4699923 , 3.45787368, 3.12725703, 2.94569389, 2.73889018,
       2.5325761 , 2.23542608, 2.46944209, 2.56960642, 2.43645342,
       2.1284713 , 1.9557629 , 1.59465591, 1.91517025, 1.59588303,
       1.47767179, 1.47288968, 1.33488903, 1.53318068, 1.6609518 ,
       1.32909511, 1.02758751, 0.84993024, 0.93306456, 0.78463831,
       0.46082139, 0.57464396, 0.25071485, 0.27407862, 0.03337467,
       0.47898398, 0.57542121, 0.73154617, 0.77017859, 0.5655132 ,
       0.61251913, 0.71646253, 0.50069004, 0.67137369, 0.29312202,
       0.63182543, 0.55160259, 0.88099891, 0.67896683, 0.7110173 ,
       0.37708521, 0.05634825, 0.00673444])